In [1]:
import pandas as pd
import teradataml as tdml

# check if data already exists

In [1]:
complaints_uploaded = False
embeddings_uploaded = False

In [ ]:
try:
    DF_complaints = tdml.DataFrame("consumer_complaints")
    print("consumer_complaints already exists")
    if DF_complaints.shape == (80771, 19):
        complaints_uploaded = True
        print("consumer_complaints has got the right shape")
        
except:
    pass
    

In [ ]:
try:
    DF_embeddings = tdml.DataFrame("consumer_embeddings")    
    print("consumer_embeddings already exists")
    if DF_embeddings.shape == (80765, 769):
        embeddings_uploaded = True
        print("consumer_embeddings has got the right shape")
except:
    pass
    

# read and upload complaints

In [ ]:
if not complaints_uploaded:
    df_complaints = pd.read_parquet("data_raw/consumer-complaints.parquet")
    df_complaints.columns = ['row_id',
         'date_received',
         'product',
         'subproduct',
         'issue',
         'subissue',
         'consumer_complaint_narrative',
         'company_public_response',
         'company',
         'state',
         'zip_code',
         'tags',
         'consumer_consent_provided',
         'submitted_via',
         'date_sent_to_company',
         'company_response_to_consumer',
         'timely_response',
         'consumer_disputed',
         'complaint_id']
    complaints_uploaded

    df_complaints["date_received"] = pd.to_datetime(df_complaints.date_received)
    df_complaints["date_sent_to_company"] = pd.to_datetime(df_complaints.date_sent_to_company)
    print("start upload: consumer_complaints")
    tdml.copy_to_sql(df_complaints, "consumer_complaints", 
                     types = {"consumer_complaint_narrative":tdml.VARCHAR(40000),
                             "zip_code":tdml.VARCHAR(10),
                             "date_received":tdml.DATE(),
                              "date_sent_to_company":tdml.DATE(),
                             },
                     primary_index="row_id",
                     if_exists="fail"
                    )
    print("finished upload: consumer_complaints")

# read and upload embeddings 

In [ ]:
if not embeddings_uploaded:
    list_dfs = []
    for i in range (1,10):
        list_dfs.append(pd.read_parquet(f"data_embeddings/embeddings_{i}.parquet"))
    df_embeddings = pd.concat(list_dfs)
    print("start upload: consumer_embeddings")
    tdml.copy_to_sql(df_embeddings, "consumer_embeddings", 
                 primary_index="row_id",
                 if_exists="fail"
                )
    print("finished upload: consumer_embeddings")